In [ ]:
import json
import streamlit as st
import numpy as np
import pandas as pd
import pandas_bokeh
import plotly.express as px

### 导入数据

In [ ]:
### 文件路径D:\Jupyter\Plotly绘图
Path1 = 'D:/Jupyter/Plotly绘图/原始数据/订单数据.csv'
Path2 = 'D:/Jupyter/Plotly绘图/原始数据/Kadehome商品总表.xlsx'
Path3 = 'D:/Jupyter/Plotly绘图/原始数据/国家洲名称转换.xlsx'
### 读取店铺数据到Dataframe
df_order = pd.read_csv(Path1)
df_items = pd.read_excel(Path2)
df_Statename = pd.read_excel(Path3)
### 合并处理数据，并转换时间序列
df_merge = pd.merge(df_order, df_items, on="Item Number", how="left")
df_merge['总销售额'] = df_merge['Quantity']*df_merge['Wholesale Price']
df_merge['订单时间'] = pd.to_datetime(df_merge['PO Date'],format='%m/%d/%Y')

### 导出标准化Dataframe
df_sel = df_merge[
    ['PO Number', '订单时间', 'Order Status','分类','Item Number', '唯非SKU','供应商SKU','供应商编码','Wholesale Price',
     'Quantity','总销售额', 'Ship To State']
    ] 
df_use = df_sel.rename(
    columns={'PO Number':'订单号',
             'Order Status':'订单状态',
             'Item Number':'平台SKU',
             'Wholesale Price':'销售价格',
             'Quantity':'销售数量',
             'Ship To State':'state'}
    )
df_useful = df_use.dropna()
df_useful.head()

### 绘制仪表盘数据

In [ ]:
a = df_useful['总销售额'].sum()
print('%.2f' %a)

In [ ]:
df_orderstatus = df_use['订单状态'].value_counts().rename_axis('status').reset_index(name='counts')
df_orderstatus
ordersum = df_orderstatus['counts'].sum()
ordersum
df_orderstatus
c =  df_orderstatus.drop(
    index = df_orderstatus[(df_orderstatus.status == 'Cancelled')].index.tolist()
    ) # 删除Cancelled订单
c

### 开始绘制地图

In [ ]:
file = open('D:/美国洲分布json文件/features.geojson','r',encoding='utf-8') #### 绘制地图专用
counties = json.load(file)

In [ ]:
df_State = df_useful.state.value_counts().rename_axis('state').reset_index(name='counts') #### 地图数据处理
df_State_useful = pd.merge(df_State, df_Statename, on="state", how="left")
df_State_useful.head()

In [ ]:
px.set_mapbox_access_token(token=st.secrets["token"])
fig = px.choropleth_mapbox(df_State_useful, geojson=counties, locations='statename', color='counts',
                           color_continuous_scale="Viridis",
                           range_color=(0, 200),
                           mapbox_style="streets",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# fig2 = px.treemap(df_useful, path=[px.Constant("总计"),'分类', '供应商编码','唯非SKU'], values='总销售额',
#                   color='总销售额', hover_data=['总销售额'],
#                   color_continuous_scale='RdBu',
#                   color_continuous_midpoint=np.average(df_useful['总销售额'], weights=df_useful['销售数量']))
# fig2.update_layout(margin = dict(t=50, l=25, r=25, b=25))

# df1 = df_use.groupby(['订单时间'],as_index=False)['总销售额'].sum()
# df1.head()
# fig3 = px.line(df1, x="订单时间", y="总销售额", title='销售情况')

# st.write(fig2)
# st.write(fig3)

### 开始绘制折线图

In [ ]:
pandas_bokeh.output_notebook()
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=list('ABCD'))
df = df.cumsum()

df.plot_bokeh(rangetool=True)

In [ ]:
df1 = df_use.groupby(['订单时间'],as_index=False)['总销售额'].sum()
df1.plot_bokeh.line(
    title="销售情况",
    x = '订单时间',
    y = '总销售额',
    xlabel="Date",
    ylabel="price [$]",
    yticks=[0, 5000],
    ylim=(0, 5000),
    colormap=["blue"],
    rangetool=True,
    marker="asterisk")

In [68]:
fig2 = px.treemap(df_useful, path=[px.Constant("总计"),'分类', '供应商编码','唯非SKU'], values='总销售额',
                  color='总销售额', hover_data=['总销售额'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=2,)
fig2.update_layout(margin = dict(t=50, l=25, r=25, b=25))

fig2

In [50]:
df = px.data.gapminder().query("year == 2007")
df.head()
# a = np.average(df_useful['总销售额'], weights=df_useful['总销售额'])
b = np.average(df['lifeExp'], weights=df['pop'])
print(b)

68.91909251904043


In [58]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/sales_success.csv')
print(df.head())

levels = ['salesperson', 'county', 'region'] # levels used for the hierarchical chart
color_columns = ['sales', 'calls']
value_column = 'calls'

def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'total'
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]] / dfg[color_columns[1]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='total', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns[0]].sum() / df[color_columns[1]].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees


df_all_trees = build_hierarchical_dataframe(df, levels, value_column, color_columns)
average_score = df['sales'].sum() / df['calls'].sum()

fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],)

fig.add_trace(go.Treemap(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}',
    name=''
    ), 1, 1)

fig.add_trace(go.Treemap(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}',
    maxdepth=2
    ), 1, 2)

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

   Unnamed: 0 region   county salesperson  calls  sales
0           0  North   Dallam          JE     35     23
1           1  North   Dallam          ZQ     49     13
2           2  North   Dallam          IJ     20      6
3           3  North  Hartley          WE     39     37
4           4  North  Hartley          PL     42     37


C:\Windows\Temp\ipykernel_15500\2849239227.py:31: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Windows\Temp\ipykernel_15500\2849239227.py:31: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Windows\Temp\ipykernel_15500\2849239227.py:31: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Windows\Temp\ipykernel_15500\2849239227.py:35: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [62]:
def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'total'
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]] / dfg[color_columns[1]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='total', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns[0]].sum() / df[color_columns[1]].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees

df_all_trees.head()

,id,parent,value,color
0,AX,Houston,42,0.214286
1,BI,Hartley,24,0.333333
2,BS,Brewster,33,0.424242
3,DQ,Presidio,40,0.375000
4,DS,Duval,42,0.214286


In [66]:
a = fig.add_trace(go.Treemap(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}',
    maxdepth=2
    ), 1, 2)


a

In [ ]:
fig2 = px.treemap(df_useful, path=[px.Constant("总计"),'分类', '供应商编码','唯非SKU'], values='总销售额',
                  color='总销售额', hover_data=['总销售额'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=2,)
fig2.update_layout(margin = dict(t=50, l=25, r=25, b=25))

In [74]:
import plotly.graph_objects as go

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/96c0bd/sunburst-coffee-flavors-complete.csv')

df

,ids,labels,parents
0,Coffee,Coffee Flavors,NaN
1,Aromas,Aromas,Coffee
2,Tastes,Tastes,Coffee
3,Aromas-Enzymatic,Enzymatic,Aromas
4,Aromas-Sugar Browning,Sugar Browning,Aromas
...,...,...,...
92,Pungent-Thyme,Thyme,Spicy-Pungent
93,Smokey-Tarry,Tarry,Carbony-Smokey
94,Smokey-Pipe Tobacco,Pipe Tobacco,Carbony-Smokey
95,Ashy-Burnt,Burnt,Carbony-Ashy


In [72]:
import plotly.graph_objects as go

import pandas as pd

df = df_useful

fig = go.Figure()

fig.add_trace(go.Treemap(
    ids = df_useful.唯非SKU,
    labels = df_useful.供应商编码,
    parents = df_useful.分类,
    maxdepth=3,
    root_color="lightgrey"
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))

fig.show()